# Useful links

### Getting started

Titanic problem  
https://www.kaggle.com/c/titanic

Tutorial with python  
https://www.kaggle.com/c/titanic/details/getting-started-with-python

Tutorial with pandas in python  
https://www.kaggle.com/c/titanic/details/getting-started-with-python-ii 

Getting started with random forests  
https://www.kaggle.com/c/titanic/details/getting-started-with-random-forests 